## 4.1 手动访问迭代器中的元素

### 问题

我们需要处理某个可迭代对象中的元素，但是基于某种原因不能也不想使用 for 循环

### 解决方案

使用 next() 函数，然后自己编写代码来捕获 StopIteration 异常

In [1]:
items = [1, 2, 3]
it = iter(items)
try:
    while True:
        line = next(it)
        print(line, end='')
except StopIteration:
    pass

123

In [3]:
it = iter(items)  # Invokes items.__iter__()
next(it)  # Invokes it.__next__()

1

## 4.2 委托迭代

### 问题

我们构建了一个自定义的容器对象，其内部持有一个列表、元组或其他的可迭代对象。我们想让自己的新容器能够完成迭代操作

### 解决方案

一般来说，我们所要做的就是定义一个 \_\_iter\_\_() 方法，将迭代请求委托到对象内部持有的容器上

In [8]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
    
    def __iter__(self):
        return iter(self._children)
    
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    for ch in root:
        print(ch)

Node(1)
Node(2)


### 讨论

Python 的迭代协议要求 \_\_iter\_\_() 返回一个特殊的迭代器对象，由该对象实现的 \_\_next\_\_() 方法来完成实际的迭代。如果要做的只是迭代另一个容器中的内容，我们不必担心底层细节是如何工作的，所要做的就是转发迭代请求。

## 4.3 用生成器创建新的迭代模式

### 问题

我们想实现一个自定义的迭代模式，使其区别于常见的内建函数（即 range()、reversed() 等）

### 解决方案

可使用生成器函数来定义

In [10]:
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x
        x += increment

for n in frange(0, 4, 0.5):
    print(n)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5


In [21]:
class PrimeNumbers:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
    def isPrimeNum(self, k):
        if k < 2:
            return False
        
        for i in range(2, int(k**(1/2))+1):
            if k % i == 0:
                return False
        return True
    
    def __iter__(self):
        for k in range(self.start, self.end + 1):
            if self.isPrimeNum(k):
                yield k

for x in PrimeNumbers(1, 100):
    print(x)

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97


## 4.4 实现迭代协议

### 问题

我们正在构建一个自定义的对象，希望它可以支持迭代操作，但是也希望能有一种简单的方式来实现迭代协议

### 解决方案

要在对象上实现可迭代功能，最简单的方式就是使用生成器函数

In [22]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()
            
# Example
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))
    
    for ch in root.depth_first():
        print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


## 4.5 反向迭代

### 问题

我们想要反向迭代序列中的元素

### 解决方案

使用内建的 reversed() 函数实现反向迭代

In [23]:
a = [1, 2, 3, 4]
for x in reversed(a):
    print(x)

4
3
2
1


反向迭代只有在待处理的对象拥有可确定的大小，或者对象实现了\_\_reversed\_\_()特殊方法时，才能奏效。如果这两个条件无法满足，则必须首先将这个对象转换为列表。  
但是这样可能会消耗大量的内存，尤其是当可迭代对象较大时更是如此。  

### 讨论

如果实现了\_\_reversed\_\_()方法，那么就可以在自定义的类上实现反向迭代。示例如下：

In [24]:
class FloatRange:
    
    def __init__(self, start, end, step=0.1):
        self.start = start
        self.end = end
        self.step = step
        
    def __iter__(self):
        t = self.start
        while t <= self.end:
            yield t
            t += self.step
    
    def __reversed__(self):
        t = self.end
        while t >= self.start:
            yield t
            t -= self.step

for x in FloatRange(1.0, 4.0, 0.5):
    print(x)
    
print()
for x in reversed(FloatRange(1.0, 4.0, 0.5)):
    print(x)

1.0
1.5
2.0
2.5
3.0
3.5
4.0

4.0
3.5
3.0
2.5
2.0
1.5
1.0


## 4.6 定义带有额外状态的生成器函数

### 问题

我们想定义一个生成器函数，但是它还涉及一些额外的状态，我们希望能以某种形式将这些状态暴露给用户

### 解决方案

可以轻易地将其实现为一个类，然后把生成器函数的代码放到\_\_iter\_\_()方法中

In [25]:
from collections import deque

class linehistory:
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)
        
    def __iter__(self):
        for lineno, line in enumerate(self.lines, 1):
            self.history.append((lineno, line))
            yield line
        
    def clear(self):
        self.history.clear()
        
# TODO

In [ ]:
with open('somefile.txt') as f:
    lines = linehistory(f)
    for line in lines:
        if 'python' in line:
            for lineno, hline in lines.history:
                print(f'{lineno}:{hline}', end='')

## 4.7 对迭代器做切片操作

### 问题

我们想对由迭代器产生的数据做切片处理，但是普通的切片操作符在这里不管用

### 解决方案

要对迭代器和生成器做切片操作，itertools.islice() 函数是完美的选择

In [1]:
def count(n):
    while True:
        yield n
        n += 1

In [2]:
c = count(0)
c[10:20]

TypeError: 'generator' object is not subscriptable

In [3]:
import itertools
for x in itertools.islice(c, 10, 20):
    print(x)

10
11
12
13
14
15
16
17
18
19


### 讨论

islice() 产生的结果是一个迭代器，通过访问并丢弃所有起始索引之前的元素来实现。之后的元素会由 islice 对象产生出来，直到到达结束索引为止。

islice() 会消耗掉所提供的迭代器中的数据。由于迭代器中的元素只能访问一次，如果之后还需要倒回去访问前面的数据，应该先将数据转到列表中去。

## 4.8 跳过可迭代对象中的前一部分元素

### 问题

我们想对某个可迭代对象做迭代处理，但是对于前面几个元素并不感兴趣

### 解决方案

使用 itertools.dropwhile() 函数，提供一个函数和一个可迭代对象即可

In [4]:
with open(r'C:\Users\Ph\Jupyter_notebook\高级编程技巧\demo.txt') as f:
    for line in f:
        print(line, end='')

abcdef
xyz
123

In [5]:
from itertools import dropwhile
with open(r'C:\Users\Ph\Jupyter_notebook\高级编程技巧\demo.txt') as f:
    for line in dropwhile(lambda line: line.startswith('abc'), f):
        print(line, end='')

xyz
123

如果恰好知道要跳过多少个元素，那么可以使用 itertools.islice() 

In [6]:
from itertools import islice
items = ['a', 'b', 'c', 1, 4, 10, 15]
for x in islice(items, 3, None):
    print(x)

1
4
10
15


## 4.9 迭代所有可能的组合或排列

### 问题

我们想对一系列元素所有可能的组合或排列进行迭代

### 解决方案

为了解决这个问题，itertools 模块中提供了3个函数。第一个是 itertools.permutations()——它接受一个元素集合，将其中所有的元素重排列为所有可能的情况，并以元组序列的形式返回

In [9]:
items = ['a', 'b', 'c']
from itertools import permutations
for p in permutations(iters):
    print(p)

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')


如果想得到较短长度的所有全排列，可以提供一个可选的长度参数

In [10]:
for p in permutations(items, 2):
    print(p)

('a', 'b')
('a', 'c')
('b', 'a')
('b', 'c')
('c', 'a')
('c', 'b')


使用 itertools.combinations() 可产生输入序列中所有元素的全部组合形式

In [11]:
from itertools import combinations
for c in combinations(items, 3):
    print(c)

('a', 'b', 'c')


In [12]:
for c in combinations(items, 2):
    print(c)

('a', 'b')
('a', 'c')
('b', 'c')


In [13]:
for c in combinations(items, 1):
    print(c)

('a',)
('b',)
('c',)


若要允许相同的元素得到多次选择，则应使用 itertools.combinations_with_replacement() 函数

In [15]:
from itertools import combinations_with_replacement
for c in combinations_with_replacement(items, 3):
    print(c)

('a', 'a', 'a')
('a', 'a', 'b')
('a', 'a', 'c')
('a', 'b', 'b')
('a', 'b', 'c')
('a', 'c', 'c')
('b', 'b', 'b')
('b', 'b', 'c')
('b', 'c', 'c')
('c', 'c', 'c')


## 4.10 以索引-值对的形式迭代序列

### 问题

我们想迭代一个序列，但是又想记录下序列中当前处理到的元素索引

### 解决方案

内奸的 enumerate() 函数可以非常漂亮地解决这个问题

In [16]:
my_list = ['a', 'b', 'c']
for idx, val in enumerate(my_list):
    print(idx, val)

0 a
1 b
2 c


In [17]:
for idx, val in enumerate(my_list, start=1):
    print(idx, val)

1 a
2 b
3 c


## 4.11 同时迭代多个序列

### 问题

我们想要迭代的元素包含在多个序列中，我们想同时对它们进行迭代

### 解决方案

可以使用 zip() 函数来同时迭代多个序列

In [18]:
xpts = [1, 5, 4, 2, 10, 7]
ypts = [101, 78, 37, 15, 62, 99]
for x, y in zip(xpts, ypts):
    print(x, y)

1 101
5 78
4 37
2 15
10 62
7 99


zip(a, b) 的工作原理是创建出一个迭代器，该迭代器可产生出元组（x，y）。整个迭代的长度和其中最短的输入序列长度相同

In [19]:
zpts = ['a', 'c', 'e', 'f', 'i', 's']
for x, y, z in zip(xpts, ypts, zpts):
    print(x, y, z)

1 101 a
5 78 c
4 37 e
2 15 f
10 62 i
7 99 s


In [20]:
from itertools import zip_longest
a = [1, 2, 3]
b = ['q', 'w', 'e', 'r']
for i in zip_longest(a, b):
    print(i)

(1, 'q')
(2, 'w')
(3, 'e')
(None, 'r')


In [22]:
s = dict(zip(zpts, xpts))
print(s)

{'a': 1, 'c': 5, 'e': 4, 'f': 2, 'i': 10, 's': 7}


In [23]:
for name, val in zip(zpts, xpts):
    print(name, '=', val)

a = 1
c = 5
e = 4
f = 2
i = 10
s = 7


## 4.12 在不同的容器中进行迭代

### 问题

我们需要对许多对象执行相同的操作，但是这些对象包含在不同的容器内，而我们希望可以避免写出嵌套的循环处理，保持代码的可读性

### 解决方案

itertools.chain() 方法可以用来简化这个任务

In [27]:
from itertools import chain
a = [1, 2, 3, 4]
b = ['x', 'y', 'z']
for x in chain(a, b):
    print(x)

1
2
3
4
x
y
z


### 讨论

itertools.chain() 可接受一个或多个可迭代对象作为参数，然后它会创建一个迭代器

## 4.13 创建处理数据的管道

### 问题

我们想以流水线式的形式对数据进行迭代处理（pipeline）。比方说我们有海量的数据需要处理，但是没法完全将数据加载到内存中去

### 解决方案

生成器函数

**代码部分见书 p135** 

## 4.14 扁平化处理嵌套型的序列

### 问题

我们有一个嵌套型的序列，想将它扁平化处理为一列单独的值

### 解决方案

写一个带有 yield from 语句的递归生成器函数来解决

In [28]:
from collections import Iterable

def flatten(items, ignore_types=(str, bytes)):
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x)
        else:
            yield x
            
items = [1, 2, [3, 4, [5, 6], 7], 8]
for x in flatten(items):
    print(x)

1
2
3
4
5
6
7
8


In [29]:
items = ['Dave', 'Paula', ['Thomas', 'Lewis']]
for x in flatten(items):
    print(x)

Dave
Paula
Thomas
Lewis


### 讨论

如果想编写生成器用来把其他的生成器当成子例程调用，yield from 是个不错的快捷方式。如果不这么用，就需要编写有额外 for 循环的代码。

## 4.15 合并多个有序序列，再对整个有序序列进行迭代

### 问题

我们有一组有序序列，想对它们合并在一起之后的有序序列进行迭代

### 解决方案

heapq.merge() 函数

In [30]:
import heapq
a = [1, 4, 7, 10]
b = [2, 5, 6, 11]
for c in heapq.merge(a, b):
    print(c)

1
2
4
5
6
7
10
11


## 4.16 用迭代器取代 while 循环

### 问题

我们的代码采用 while 循环来迭代处理数据，因为这其中涉及调用某个函数或有某种不常见的测试条件，而这些东西没法归类为常见的迭代模式

### 解决方案

内建函数 iter()

In [32]:
CHUNKSIZE = 8192

def reader(s):
    while True:
        data = s.recv(CHUNKSIZE)
        if data == b'':
            break
        process_data(data)

这样的代码常常可以用 iter() 来替换

In [33]:
def reader(s):
    for chunk in iter(lambda:s.recv(CHUNKSIZE), b''):
        process_data(data)

In [36]:
import sys
f = open(r'C:\Users\Ph\Jupyter_notebook\高级编程技巧\信息.txt', encoding='utf-8')
for chunk in iter(lambda:f.read(10), ''):
    n = sys.stdout.write(chunk)

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 林清猫耳
林清猫耳  1  但行好事  莫问前程<span class="emoji emoji1f388"></span>
南  2  自由且无用
依然爱你  2 江西 
吴遥  1 福建     Walk down
毒情" 周晨 2 江西 为什么那么难过却要说我很开心
OSError(22, 'Invalid argument')
通梦。  2  晤
在一起 郑雅丽 2 江西 时光有过多少个爱人
easyfuse blossom 陈涵薇 0  <span class="emoji emoji1f490"></span>卖花的小姑娘( ˘•ω•˘ )ง提供鲜花解决方案
林伟鸿  1 福建 
陆清璃 王舒蓉 2 福建 HOW 
柱柱 吴玉柱 1  
狂野逗逗仔  0  
自渡 吴锦标 1 福建 您。
多吃一些番茄 董亚飞 1 江西 毕竟阿
泽川  0  
曹晨星  1 浙江 我就点个赞
谢佳瑜  2 福建 
奔跑吧，韩宝酱～ 韩晨 2  
张氏娱乐传媒  2  大号好友已满。小号kk000067
汤镥锴  1 江西 先让自己变得优秀，再帮助别人优秀！
愿时光温柔以待i  2  有生之年，狭路相逢，终不能幸免.
张伟伟伟YY 张伟 1 福建 总有人很无聊、   
逆流 徐泽灿 1 江西 …
疯子的鞋铺  2 江西 主营一线运动品牌鞋服    精品货源   诚招代理
HHB  1 江西 
奇奇  2  美好的追求，残缺的接纳。
占美清 占美清  0  
绿草 MF8248 2 福建 
生活就是拼博～刘炳岳  0  
撒哈拉的骄阳  1 江苏 
live 天线宝宝 0  
王先进 王安宇 1 陕西 
好福利_朋友圈  0  
Truth K 柯增项 1 Munich 
大枣18792957570  0  
Marsh 符凯凯 1  
EliaC���� 浮梦崇华 2  
bob 凌博 1 陕西 
---可可欠--- 李歌 2 陕西 Smi

### 讨论

关于内建函数 iter()，一个少有人知的特性是它可以选择性接受一个无参的可调用对象以及一个哨兵（结束）值作为输入。当以这种方式使用时，iter() 会创建一个迭代器，然后重复调用用户提供的可调用对象，直到它返回哨兵值为止。